# Nimslo Image Alignment Demo

This notebook demonstrates the complete pipeline for aligning Nimslo 4-lens camera images and generating boomerang GIFs.

## Setup and Imports

In [1]:
import sys
from pathlib import Path
import importlib
import warnings
import os

# CRITICAL: Set these BEFORE importing anything that uses OpenMP (rembg/onnxruntime)
# This prevents kernel crashes from OMP conflicts and fixes deprecated warnings
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
os.environ['OMP_NUM_THREADS'] = '1'  # Limit threads to avoid conflicts
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OMP_MAX_ACTIVE_LEVELS'] = '1'  # Use max_active_levels instead of deprecated nested

# Try to configure OpenMP programmatically
try:
    import ctypes
    try:
        # Set max_active_levels directly if possible
        libomp = ctypes.CDLL(None)
        if hasattr(libomp, 'omp_set_max_active_levels'):
            libomp.omp_set_max_active_levels(1)
            print("✓ OpenMP configured to use max_active_levels")
    except:
        pass
except:
    pass

# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', message='.*omp_set_nested.*')

# Add code directory to path
code_dir = Path().absolute()
if str(code_dir) not in sys.path:
    sys.path.insert(0, str(code_dir))

import cv2
import numpy as np

print("✓ Basic imports successful")

# Import and reload modules to pick up any changes (useful during development)
try:
    import nimslo_core.gif_generator
    importlib.reload(nimslo_core.gif_generator)
except Exception as e:
    print(f"Note: Could not reload module (this is OK on first run): {e}")

from nimslo_core import (
    preprocess_image,
    align_images,
    extract_features,
    match_features
)

# Import create_boomerang_gif AFTER reload to get latest version
from nimslo_core.gif_generator import create_boomerang_gif

# DON'T import segmentation here - it will crash the kernel
# We'll import it only when needed, or use depth-based segmentation
print("✓ Core modules loaded (segmentation deferred)")

# Verify the function has the expected parameters
import inspect
sig = inspect.signature(create_boomerang_gif)
has_normalize = 'normalize_brightness' in sig.parameters
has_strength = 'brightness_strength' in sig.parameters

print("✓ All modules loaded successfully!")
if has_normalize and has_strength:
    print("✓ Brightness normalization available (normalize_brightness, brightness_strength)")
else:
    print("⚠ Warning: Brightness normalization parameters not found - restart kernel if needed")
print("\n⚠ Note: Segmentation will be loaded on-demand to avoid kernel crashes")

✓ Basic imports successful
✓ Core modules loaded (segmentation deferred)
✓ All modules loaded successfully!
✓ Brightness normalization available (normalize_brightness, brightness_strength)

⚠ Note: Segmentation will be loaded on-demand to avoid kernel crashes


In [2]:
# Configuration
RAW_DIR = Path("../nimslo_raw")
OUTPUT_DIR = Path("../outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

BATCH_NAME = "12"  # Change this to process different batches

## Find Available Batches

In [3]:
# Find all batch directories
batch_dirs = sorted([
    d for d in RAW_DIR.iterdir()
    if d.is_dir() and (d.name.isdigit() or d.name.replace("-", "").isdigit())
])

print(f"Found {len(batch_dirs)} batches.")

Found 19 batches.


In [4]:
# Load images from selected batch
batch_path = RAW_DIR / BATCH_NAME
image_files = sorted(batch_path.glob('*.jpg')) + sorted(batch_path.glob('*.JPG'))
image_files = image_files[:4]  # Take first 4 images

# Store original images (before any processing)
original_images = []
for f in image_files:
    img = cv2.imread(str(f))
    if img is not None:
        original_images.append(img)

# Normalize sizes of originals (just cropping, no denoising/contrast adjustment)
from nimslo_core.preprocessing import normalize_sizes
original_images = normalize_sizes(original_images)

print(f"Loaded {len(original_images)} images from {BATCH_NAME}")
print(f"Original image dimensions: {original_images[0].shape}")

Loaded 4 images from 12
Original image dimensions: (2137, 1535, 3)


## Preprocessing

In [5]:
# ============================================================================
# PREPROCESSING CONFIGURATION
# ============================================================================
# Choose preprocessing approach to avoid kernel crashes:
#
# APPROACH 1: Skip denoising (safest, SIFT is robust to noise)
#   - Set USE_DENOISING = False
#   - Fastest, zero crash risk
#
# APPROACH 2: Downscale → denoise → use for alignment (safer with denoising)
#   - Set USE_DENOISING = True and SAFE_PROCESSING_SIZE = 1024
#   - Denoise runs on smaller images, then we use them for alignment
#   - Transformations are applied to full-res originals for final GIF
# ============================================================================

USE_DENOISING = False  # Set to True for Approach 2
SAFE_PROCESSING_SIZE = 1024  # Max dimension for safe denoising (only used if USE_DENOISING=True)

from nimslo_core.preprocessing import resize_for_processing

if USE_DENOISING:
    # APPROACH 2: Downscale for safe denoising
    print(f"Approach 2: Downscaling to {SAFE_PROCESSING_SIZE}px for safe denoising...")
    
    # Downscale images for processing
    small_images = []
    scales = []
    for img in original_images:
        small, scale = resize_for_processing(img, max_dimension=SAFE_PROCESSING_SIZE)
        small_images.append(small)
        scales.append(scale)
    
    print(f"  Downscaled from {original_images[0].shape[:2]} to {small_images[0].shape[:2]} (scale: {scales[0]:.2f})")
    
    # Denoise the smaller images (safe)
    preprocessed = [preprocess_image(img, denoise=True) for img in small_images]
    print(f"  ✓ Denoised {len(preprocessed)} images at safe resolution")
    
else:
    # APPROACH 1: Skip denoising entirely (safest)
    print("Approach 1: Skipping denoising (SIFT is robust to noise)...")
    
    # Just balance exposure, no denoising
    preprocessed = [preprocess_image(img, denoise=False, balance=True) for img in original_images]
    print(f"  ✓ Preprocessed {len(preprocessed)} images (exposure balanced, no denoising)")

print(f"\nPreprocessed image dimensions: {preprocessed[0].shape}")

Approach 1: Skipping denoising (SIFT is robust to noise)...
  ✓ Preprocessed 4 images (exposure balanced, no denoising)

Preprocessed image dimensions: (2137, 1535, 3)


## Subject Segmentation

In [6]:
# ============================================================================
# SEGMENTATION CONFIGURATION
# ============================================================================
# The depth model (Intel DPT) can also crash on large images.
# Options:
#
# APPROACH A: Skip segmentation, use full image (safest)
#   - Set USE_SEGMENTATION = False
#   - SIFT will find features across entire image
#   - Works well when subject is prominent
#
# APPROACH B: Use center mask (no model needed)
#   - Set USE_SEGMENTATION = False and USE_CENTER_MASK = True
#   - Assumes subject is roughly centered
#
# APPROACH C: Run depth segmentation (may crash on large images)
#   - Set USE_SEGMENTATION = True
#   - Only try this if Approach 1/2 preprocessing downscaled the images
# ============================================================================

USE_SEGMENTATION = False  # Set to True to run depth model (risky on large images)
USE_CENTER_MASK = True    # If not using segmentation, use center mask vs full image

print("Segmentation configuration:")
print(f"  USE_SEGMENTATION = {USE_SEGMENTATION}")
print(f"  USE_CENTER_MASK = {USE_CENTER_MASK}")

# Note: To use full segmentation safely, use the CLI instead:
# python nimslo_cli.py ../nimslo_raw/01/ -o test.gif

Segmentation configuration:
  USE_SEGMENTATION = False
  USE_CENTER_MASK = True


In [ ]:
# Generate masks based on configuration
masks = []

def create_center_mask(img, margin=0.15):
    """Create a mask covering the center region of the image."""
    h, w = img.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)
    y1, y2 = int(h * margin), int(h * (1 - margin))
    x1, x2 = int(w * margin), int(w * (1 - margin))
    mask[y1:y2, x1:x2] = 255
    return mask

def create_full_mask(img):
    """Create a mask covering the entire image (no masking)."""
    h, w = img.shape[:2]
    return np.ones((h, w), dtype=np.uint8) * 255

if USE_SEGMENTATION:
    # APPROACH C: Run depth segmentation (may crash on large images)
    print("Running depth-based segmentation (may be slow/crash on large images)...")
    from nimslo_core.segmentation import segment_subject
    
    for i, img in enumerate(preprocessed):
        try:
            mask, conf = segment_subject(img, method="depth", return_confidence=True)
            masks.append(mask)
            print(f"  Frame {i+1}: confidence={conf:.2f}, method=depth")
        except Exception as e:
            print(f"  Frame {i+1}: Error - {e}, using center mask fallback")
            masks.append(create_center_mask(img))
else:
    # APPROACH A or B: Skip segmentation model
    if USE_CENTER_MASK:
        print("Using center masks (no segmentation model)...")
        for i, img in enumerate(preprocessed):
            masks.append(create_center_mask(img))
            print(f"  Frame {i+1}: center mask (15% margin)")
    else:
        print("Using full image masks (no masking)...")
        for i, img in enumerate(preprocessed):
            masks.append(create_full_mask(img))
            print(f"  Frame {i+1}: full image mask")

print(f"\n✓ Generated {len(masks)} masks")

Frame 1: Error - name 'segment_subject' is not defined
Frame 1: Using fallback mask (center region)
Frame 2: Error - name 'segment_subject' is not defined
Frame 2: Using fallback mask (center region)
Frame 3: Error - name 'segment_subject' is not defined
Frame 3: Using fallback mask (center region)
Frame 4: Error - name 'segment_subject' is not defined
Frame 4: Using fallback mask (center region)


In [ ]:
# Visualize masks overlaid on images
# Define visualize_mask inline to avoid importing segmentation module (which may crash)
import matplotlib.pyplot as plt

def visualize_mask_simple(img, mask, alpha=0.4, color=(0, 255, 0)):
    """Overlay a mask on an image with transparency."""
    overlay = img.copy()
    mask_bool = mask > 127
    overlay[mask_bool] = (
        (1 - alpha) * overlay[mask_bool] + 
        alpha * np.array(color)
    ).astype(np.uint8)
    return overlay

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Top row: original preprocessed images
for i, (ax, img) in enumerate(zip(axes[0], preprocessed)):
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax.set_title(f"Frame {i+1} (Preprocessed)")
    ax.axis('off')

# Bottom row: images with masks overlaid
mask_type = "center" if USE_CENTER_MASK else ("depth" if USE_SEGMENTATION else "full")
for i, (ax, img, mask) in enumerate(zip(axes[1], preprocessed, masks)):
    overlay = visualize_mask_simple(img, mask, alpha=0.3, color=(0, 255, 0))
    ax.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    ax.set_title(f"Frame {i+1} ({mask_type} mask)")
    ax.axis('off')

plt.suptitle("Subject Segmentation Results", fontsize=14, y=0.995)
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

## Feature Extraction and Matching

In [ ]:
# Extract features from first two frames
kp1, des1 = extract_features(preprocessed[0], mask=masks[0], n_features=1000)
kp2, des2 = extract_features(preprocessed[1], mask=masks[1], n_features=1000)

print(f"Frame 1: {len(kp1)} keypoints, {des1.shape[0] if des1 is not None else 0} descriptors")
print(f"Frame 2: {len(kp2)} keypoints, {des2.shape[0] if des2 is not None else 0} descriptors")

# Match features (match_features already applies ratio test)
matches = match_features(des1, des2)

print(f"\nMatches after ratio test: {len(matches)}")
if len(matches) > 0:
    distances = [m.distance for m in matches]
    print(f"  Distance range: {min(distances):.2f} - {max(distances):.2f}")
    print(f"  Mean distance: {np.mean(distances):.2f}")

In [ ]:
# Visualize keypoints detected on each image
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Frame 1: Show keypoints
img1_kp = cv2.drawKeypoints(
    preprocessed[0], kp1, None,
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
axes[0].imshow(cv2.cvtColor(img1_kp, cv2.COLOR_BGR2RGB))
axes[0].set_title(f"Frame 1: {len(kp1)} keypoints")
axes[0].axis('off')

# Frame 2: Show keypoints
img2_kp = cv2.drawKeypoints(
    preprocessed[1], kp2, None,
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
axes[1].imshow(cv2.cvtColor(img2_kp, cv2.COLOR_BGR2RGB))
axes[1].set_title(f"Frame 2: {len(kp2)} keypoints")
axes[1].axis('off')

plt.suptitle("Feature Keypoints Detection", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
# Visualize matches (use matches from match_features, not filtered)
if len(matches) > 0:
    # Sort by distance to show best matches first
    sorted_matches = sorted(matches, key=lambda x: x.distance)
    
    img_matches = cv2.drawMatches(
        preprocessed[0], kp1,
        preprocessed[1], kp2,
        sorted_matches[:50],  # Show best 50 matches
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )
    
    plt.figure(figsize=(16, 8))
    plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.title(f"Feature Matches Between Frame 1 and Frame 2 (showing best {min(50, len(matches))} of {len(matches)})")
    plt.axis('off')
    plt.show()
else:
    print("No matches found! This could indicate:")
    print("  - Masks are too restrictive (not enough overlap)")
    print("  - Images are too different")
    print("  - Feature extraction failed")
    
    # Try without masks to see if that helps
    print("\nTrying without masks...")
    kp1_no_mask, des1_no_mask = extract_features(preprocessed[0], mask=None, n_features=1000)
    kp2_no_mask, des2_no_mask = extract_features(preprocessed[1], mask=None, n_features=1000)
    matches_no_mask = match_features(des1_no_mask, des2_no_mask)
    print(f"Without masks: {len(matches_no_mask)} matches")
    
    if len(matches_no_mask) > 0:
        sorted_matches = sorted(matches_no_mask, key=lambda x: x.distance)
        img_matches = cv2.drawMatches(
            preprocessed[0], kp1_no_mask,
            preprocessed[1], kp2_no_mask,
            sorted_matches[:50],
            None,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )
        plt.figure(figsize=(16, 8))
        plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
        plt.title(f"Matches WITHOUT masks (showing best {min(50, len(matches_no_mask))} of {len(matches_no_mask)})")
        plt.axis('off')
        plt.show()

## Image Alignment

In [ ]:
# Align preprocessed images (for good feature matching)
# This gives us the transformation matrices
aligned_preprocessed, alignment_results = align_images(
    preprocessed, masks,
    n_features=1000
)

print("Alignment results:")
for i, result in enumerate(alignment_results):
    if i > 0:  # Skip reference frame
        print(f"Frame {i+1}: {result.total_matches} matches, {result.inliers} inliers, IoU: {result.iou:.2f}")

# Apply the same transformations to original images (max quality)
print("\nApplying transformations to original images...")
aligned_originals = []
ref_h, ref_w = original_images[0].shape[:2]

# Check if we need to scale the homography (Approach 2: preprocessed at different resolution)
preproc_h, preproc_w = preprocessed[0].shape[:2]
need_scale = (preproc_h != ref_h) or (preproc_w != ref_w)

if need_scale:
    # Scale factors from preprocessed to original
    scale_x = ref_w / preproc_w
    scale_y = ref_h / preproc_h
    print(f"  Scaling transforms from {preproc_w}x{preproc_h} to {ref_w}x{ref_h}")
    
    # Scaling matrices: S scales up, S_inv scales down
    S = np.array([[scale_x, 0, 0], [0, scale_y, 0], [0, 0, 1]], dtype=np.float64)
    S_inv = np.array([[1/scale_x, 0, 0], [0, 1/scale_y, 0], [0, 0, 1]], dtype=np.float64)

for i, (orig_img, result) in enumerate(zip(original_images, alignment_results)):
    if i == 0:
        # Reference frame stays as-is
        aligned_originals.append(orig_img.copy())
    else:
        # Apply transformation from alignment result
        # transform is stored as 3x3 (affine padded or homography)
        transform = result.transform
        
        if need_scale:
            # Scale homography: H_full = S * H_small * S_inv
            transform = S @ transform @ S_inv
        
        aligned_originals.append(cv2.warpPerspective(orig_img, transform, (ref_w, ref_h)))

print("✓ Original images aligned (max quality, no denoising/contrast adjustment)")

In [ ]:
# Display aligned original images (max quality)
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, (ax, img) in enumerate(zip(axes, aligned_originals)):
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax.set_title(f"Aligned Original Frame {i+1}")
    ax.axis('off')
plt.tight_layout()
plt.show()

## Generate Boomerang GIF

In [ ]:
# Create GIF from aligned original images (max quality, no denoising/contrast adjustment)
# Optionally resize for web-friendly size (or use full resolution for max quality)
from nimslo_core.gif_generator import resize_for_web

# For max quality, use full resolution. For web-friendly, uncomment the resize line:
aligned_originals = resize_for_web(aligned_originals, max_dimension=600)

# Create boomerang GIF from original images
# crop_valid_region=True removes black bars from stereoscopic alignment
# normalize_brightness=True prevents flashing from exposure differences
# brightness_strength controls how much correction (0.0-1.0, default 0.5 = moderate)
output_path = OUTPUT_DIR / f"test_{BATCH_NAME}.gif"
gif_path = create_boomerang_gif(
    aligned_originals, 
    output_path,
    crop_valid_region=True,  # Remove black borders from warping
    normalize_brightness=True,  # Equalize brightness to prevent flashing
    brightness_strength=0.5  # Moderate correction (0.0 = none, 1.0 = full)
)

print(f"✓ GIF saved to: {gif_path}")
print(f"  File size: {gif_path.stat().st_size / 1024:.1f} KB")
print(f"  Using original images (max quality, no denoising/contrast adjustment)")
print(f"  Cropped to remove black bars from alignment")
print(f"  Brightness normalized across frames to prevent flashing")

In [ ]:
# Display the GIF (if in Jupyter)
from IPython.display import Image, display
display(Image(str(gif_path)))